# Convolutional Nerual Network (CNN)
##### Maxwell Rosenzweig

The convolutional neural network (CNN) is designed using Tensorflow. Tensorflow and it's prerequisites must be installed (see https://www.tensorflow.org/install/pip). For HP tuning, the keras tuning module must be separately installed (see https://keras.io/keras_tuner/).

Before running this file, you need to run `demo.m` and `preprocessing.ipynb` to generate `spect_train.npy`, `downsampled_acc_train.npy`, `spect_test.npy`, and `downsampled_acc_test.npy`.

In [ ]:
import tensorflow as tf

from tensorflow.keras import *
import numpy as np
import matplotlib.pyplot as plt

import keras_tuner

In [ ]:
# Verify tensorflow install and gpu compatability
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.config.list_physical_devices('GPU'))

### Load and format testing data

Run these 2 blocks in order to load and format the training data. The 3rd block clears the original copy from memory for performance improvements. The 4 blocks after that do the same, but for testing data.

In [ ]:
# Load training data
spect_train = np.load('../data/spect_train.npy', allow_pickle=True)
acc_train = np.load('../data/downsampled_acc_train.npy', allow_pickle=True)

In [ ]:
# Reshape Training data
# Align the spectograms and acc data
# We skip the first 10 samples to get stable LFP data

stacked = np.stack(acc_train[0]).transpose()
spect_train_concat = spect_train[0][0:min(len(spect_train[0]), len(stacked))-1:]
acc_train_concat = stacked[0:min(len(spect_train[0]), len(stacked))-1:]
for i in range(1, len(spect_train)):
    # print(f'Dataset {i} has {len(spect_train[i])} inputs and {len(np.stack(acc_train[i]).transpose())} outputs')
    # print(f'acc shape is {np.shape(np.stack(acc_train[i]))}')
    stacked = np.stack(acc_train[i]).transpose()
    new_spect = spect_train[i][0:min(len(spect_train[i]), len(stacked))-1:]
    stacked = stacked[0:min(len(spect_train[i]), len(stacked))-1:]
    # throw out any data with outliers or too few data points
    if (len(new_spect) == 0):
        print(f'Dataset {i} has only {len(spect_train[i])} inputs and {len(acc_train[i].transpose())} outputs')
    elif (np.amax(new_spect) > 2e-6):
        print(f'Dataset {i} contains an outlier.')
    else:
        spect_train_concat = np.concatenate((spect_train_concat, new_spect))
        acc_train_concat = np.concatenate((acc_train_concat, stacked))

spect_train1 = spect_train_concat[...,0]
spect_train2 = spect_train_concat[...,1]

# Bring values to near [0,1.0] range
spect_train1 = (spect_train1 * 5e5)
spect_train2 = (spect_train2 * 5e5)

print(np.shape(spect_train1))
print(np.shape(spect_train2))
print(np.shape(acc_train_concat))


In [ ]:
#Sanity checks

print(np.median(spect_train1), np.amax(spect_train1), np.amin(spect_train1), np.std(spect_train1))
print(np.median(spect_train2), np.amax(spect_train2), np.amin(spect_train2), np.std(spect_train2))
print(np.median(acc_train_concat), np.amax(acc_train_concat), np.amin(acc_train_concat), np.std(acc_train_concat))

assert not np.any(np.isnan(spect_train1))
assert not np.any(np.isnan(spect_train2))
assert not np.any(np.isnan(acc_train_concat))

In [ ]:
import gc

# Free up memory from raw inputs
del spect_train
del acc_train
gc.collect()

In [ ]:
# Load testing data
spect_test = np.load('../data/spect_test.npy', allow_pickle=True)
acc_test = np.load('../data/downsampled_acc_test.npy', allow_pickle=True)

In [ ]:
# Reshape testing data
# Throw away first 7 acc values; these correspond to the data points before 400ms

stacked = np.stack(acc_test[0]).transpose()
spect_test_concat = spect_test[0][:min(len(spect_test[0]), len(stacked))-1:]
acc_test_concat = stacked[:min(len(spect_test[0]), len(stacked))-1:]
for i in range(1, len(spect_test)):
    # print(f'Dataset {i} has {len(spect_test[i])} inputs and {len(np.stack(acc_test[i]).transpose())} outputs')
    # print(f'acc shape is {np.shape(np.stack(acc_test[i]))}')
    stacked = np.stack(acc_test[i]).transpose()
    new_spect = spect_test[i][:min(len(spect_test[i]), len(stacked))-1:]
    stacked = stacked[:min(len(spect_test[i]), len(stacked))-1:]
    # throw out any data with outliers or too few data points
    if (len(new_spect) == 0):
        print(f'Dataset {i} has only {len(spect_test[i])} inputs and {len(acc_test[i].transpose())} outputs')
    elif (np.amax(new_spect) > 2e-6):
        print(f'Dataset {i} contains an outlier.')
    else:
        spect_test_concat = np.concatenate((spect_test_concat, new_spect))
        acc_test_concat = np.concatenate((acc_test_concat, stacked))

spect_test1 = spect_test_concat[...,0]
spect_test2 = spect_test_concat[...,1]
spect_test1 = (spect_test1 * 5e5)
spect_test2 = (spect_test2 * 5e5)

[spect_test1, spect_val1] = np.array_split(spect_test1, 2)
[spect_test2, spect_val2] = np.array_split(spect_test2, 2)
[acc_test_concat, acc_val_concat] = np.array_split(acc_test_concat, 2)
print(np.shape(spect_test1))
print(np.shape(spect_val1))
print(np.shape(spect_test2))
print(np.shape(spect_val2))
print(np.shape(acc_test_concat))
print(np.shape(acc_val_concat))

In [ ]:
#Sanity checks

print(np.median(spect_test1), np.amax(spect_test1), np.amin(spect_test1), np.std(spect_test1))
print(np.median(spect_test2), np.amax(spect_test2), np.amin(spect_test2), np.std(spect_test2))
print(np.median(acc_test_concat), np.amax(acc_test_concat), np.amin(acc_test_concat), np.std(acc_test_concat))

assert not np.any(np.isnan(spect_test1))
assert not np.any(np.isnan(spect_test2))
assert not np.any(np.isnan(acc_test_concat))

In [ ]:
import gc

# Free up memory from raw inputs
del spect_test
del acc_test
gc.collect()

### Define the Model
Running this block will reset any training progress. Only run if you do not want to continue to test/fit the current model.

In [ ]:
channel1 = Input(shape=(51, 46, 1))
channel2 = Input(shape=(51, 46, 1))

cnn1 = layers.Conv2D(filters=16, kernel_size=(1,46), activation='relu')(channel1)
cnn1 = layers.Conv2D(filters=32, kernel_size=(3,1), activation='relu')(cnn1)
cnn1 = layers.MaxPool2D(pool_size=(2,1))(cnn1)
cnn1 = layers.Conv2D(filters=32, kernel_size=(3,1), activation='relu')(cnn1)
cnn1 = layers.MaxPool2D(pool_size=(2,1))(cnn1)
cnn1 = layers.Flatten()(cnn1)

cnn2 = layers.Conv2D(filters=16, kernel_size=(1,46), activation='relu')(channel2)
cnn2 = layers.Conv2D(filters=32, kernel_size=(3,1), activation='relu')(cnn2)
cnn2 = layers.MaxPool2D(pool_size=(2,1))(cnn2)
cnn2 = layers.Conv2D(filters=32, kernel_size=(3,1), activation='relu')(cnn2)
cnn2 = layers.MaxPool2D(pool_size=(2,1))(cnn2)
cnn2 = layers.Flatten()(cnn2)


output = layers.concatenate([cnn1, cnn2])
output = layers.Dense(512, activation='relu')(output)
output = layers.Dense(256, activation='relu')(output)
output = layers.Dense(128, activation='relu')(output)
output = layers.Dense(3)(output)

model = models.Model(inputs=[channel1, channel2], outputs=output)
opt = optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='mse')

model.summary()

### Fit model
Run to fit the model. The second block will display the training history after the model has finished.

In [ ]:
from datetime import *

earlystop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)
earlystop2 = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=100)
history = model.fit(
    (spect_train1, spect_train2),
    acc_train_concat,
    epochs=100,
    verbose=1,
    callbacks=[earlystop, earlystop2],
    validation_data=((spect_val1, spect_val2), acc_val_concat),
    shuffle=True
)

In [ ]:
# Loss history plot
plt.plot(history.history['loss'][:])
plt.plot(history.history['val_loss'][:])
plt.ylabel('MSE')
plt.xlabel('epoch')
plt.legend(['train', 'validation'])
plt.title('Model error')

### Test Model
Optionally run the first block to load a model from file. The second block will run the model on the testing set. The following 3 will display the predicitons vs the true values of the accelerometer data.

In [ ]:
model = models.load_model('model.h5')

In [ ]:
model.evaluate((spect_test1, spect_test2), acc_test_concat, batch_size=256)
prediciton = model.predict((spect_test1, spect_test2))
prediction = np.array(prediciton)

In [ ]:
data_x_test = acc_test_concat.T[0]
data_x_pred = prediction[...,0]

plt.plot(data_x_pred, label='x prediciton')
plt.plot(data_x_test, label='x test')
plt.title("X Prediction vs Test")
plt.xlabel('Time')
plt.ylabel('Voltage')
plt.legend()
plt.show()

In [ ]:

data_y_test = acc_test_concat.T[1]
data_y_pred = prediction[...,1]
plt.plot(data_y_pred, label='y prediciton')
plt.plot(data_y_test, label='y test')
plt.title("Y Prediction vs Test")
plt.xlabel('Time')
plt.ylabel('Voltage')
plt.legend()
plt.show()

In [ ]:

data_z_test = acc_test_concat.T[2]
data_z_pred = prediction[...,2]
plt.plot(data_z_pred, label='z prediciton')
plt.plot(data_z_test, label='z test')
plt.title("Z Prediction vs Test")
plt.xlabel('Time')
plt.ylabel('Voltage')
plt.legend()
plt.show()

### Tune Hyperparameters
Be sure to import both the training and testing set to run this. Takes a long time, recommend leaving overnight.

In [ ]:
from datetime import *

def build_model(hp):

    channel1 = Input(shape=(51, 46, 1))
    channel2 = Input(shape=(51, 46, 1))

    input_drop_rate = hp.Float(f'dropout_conv_rate', 0, 0.2, step=0.05, default=0.1)
    cnn1 = layers.Dropout(input_drop_rate)(channel1)
    cnn2 = layers.Dropout(input_drop_rate)(channel2)

    hp_filters = hp.Int('filters_vert', 8, 64, step=8, default=32)
    hp_kernel_size = hp.Int('kernel_size_vert', 3, 46, step=1, default=32)
    cnn1 = layers.Conv2D(filters=hp_filters, kernel_size=(1,hp_kernel_size), activation='relu')(cnn1)
    cnn2 = layers.Conv2D(filters=hp_filters, kernel_size=(1,hp_kernel_size), activation='relu')(cnn2)

    for i in range(hp.Int('conv_layers', 1, 3, default=2)):
        hp_filters = hp.Int(f'filters_{i}', 8, 64, step=8, default=32)
        hp_kernel_size = hp.Int(f'kernel_size_{i}', 3, 5, step=2, default=3)
        cnn1 = layers.Conv2D(
            filters=hp_filters,
            kernel_size=(hp_kernel_size, 1),
            activation='relu',
            padding='same')(cnn1)
        cnn2 = layers.Conv2D(
            filters=hp_filters,
            kernel_size=(hp_kernel_size, 1),
            activation='relu',
            padding='same')(cnn2)
        cnn1 = layers.MaxPooling2D(pool_size=(2,1))(cnn1)
        cnn2 = layers.MaxPooling2D(pool_size=(2,1))(cnn2)

    conv_drop_rate = hp.Float(f'dropout_conv_rate', 0, 0.2, step=0.05, default=0.1)
    cnn1 = layers.Dropout(conv_drop_rate)(cnn1)
    cnn2 = layers.Dropout(conv_drop_rate)(cnn2)
    cnn1 = layers.Flatten()(cnn1)
    cnn2 = layers.Flatten()(cnn2)

    output = layers.concatenate([cnn1, cnn2])
    dense_units = hp.Int(f'dense_units', 64, 768, step=64, default=512)
    dense_drop_rate = hp.Float(f'dropout_dense_rate_{i}', 0, 0.5, step=0.1, default=0.3)
    for i in range(hp.Int('dense_layers', 1, 5, default=3)):
        output = layers.Dense(units=dense_units, activation='relu')(output)
        output = layers.Dropout(dense_drop_rate)(output)
    output = layers.Dense(3)(output)

    model = models.Model(inputs=[channel1, channel2], outputs=output)
    opt = optimizers.Adam(learning_rate=hp.Float('learning_rate',min_value=0.0005, max_value=0.005, sampling='log', default=0.001))
    model.compile(optimizer='adam', loss='mse')

    return model


tuner = keras_tuner.RandomSearch(
    build_model,
    max_trials=500,
    objective="val_loss"
)

logs = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs)
earlystop_loss = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)
earlystop_val_loss = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=100)

tuner.search(
    (spect_train1, spect_train2), acc_train_concat,
    epochs=300,
    validation_data=((spect_val1, spect_val2), acc_val_concat),
    shuffle=True,
    callbacks=[earlystop_loss, earlystop_val_loss, tboard_callback]
)